In [7]:
import concurrent.futures
import pandas as pd
import tensorflow as tf
import time

# Initialize TPU if needed
def init_tpu():
    try:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print("TPU initialized.")
        return True
    except Exception as e:
        print(f"TPU initialization failed: {e}")
        return False

# Load keywords and resource needs from Excel file
def load_keywords_from_excel(file_path):
    try:
        df = pd.read_excel(file_path)
        df.columns = df.columns.str.strip()  # Strip whitespace from column names
        keywords = [(row['Keyword'], int(row['CPU']), int(row['GPU']), int(row['TPU'])) for _, row in df.iterrows()]
        print(f"Loaded keywords: {keywords}")
        return keywords
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Define CNN model
def create_cnn_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# Compile and train CNN model
def compile_and_train_model(model, train_images, train_labels, epochs=10):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_images, train_labels, epochs=epochs)

# Evaluate CNN model
def evaluate_model(model, test_images, test_labels):
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
    print(f'\nTest accuracy: {test_acc}')
    return test_acc

# Load and preprocess CIFAR-10 dataset
def load_and_preprocess_data():
    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
    train_images = train_images.astype('float32') / 255.0
    test_images = test_images.astype('float32') / 255.0
    return train_images, train_labels, test_images, test_labels

# Execute a CNN snippet on the specified device and measure execution time
def run_snippet(snippet, device):
    print(f"Executing snippet on {device}")  # Print device info
    start_time = time.time()
    accuracy = None

    with tf.device(device):
        if snippet["action"] == "create_model":
            return create_cnn_model((32, 32, 3))
        elif snippet["action"] == "train_model":
            compile_and_train_model(snippet["model"], snippet["train_images"], snippet["train_labels"])
        elif snippet["action"] == "evaluate_model":
            accuracy = evaluate_model(snippet["model"], snippet["test_images"], snippet["test_labels"])

    execution_time = time.time() - start_time
    return f"Execution completed in {execution_time:.2f} seconds on {device}. Accuracy: {accuracy}" if accuracy else f"Execution completed in {execution_time:.2f} seconds on {device}."

# Decide the device based on resource availability
def get_device(cpu, gpu, tpu):
    if tpu and tf.config.list_logical_devices('TPU'):
        return '/device:TPU:0'
    elif gpu and tf.config.list_physical_devices('GPU'):
        return '/device:GPU:0'
    else:
        return '/device:CPU:0'

# Main master function for scheduling tasks
def master(file_path):
    start_time = time.time()
    keywords = load_keywords_from_excel(file_path)
    if keywords is None:
        print("Failed to load keywords.")
        return

    # Load and preprocess data for training and evaluation
    train_images, train_labels, test_images, test_labels = load_and_preprocess_data()

    # Define CNN snippets with actions
    cnn_snippets = [
        {"action": "create_model"},
        {"action": "train_model", "model": create_cnn_model((32, 32, 3)), "train_images": train_images, "train_labels": train_labels},
        {"action": "evaluate_model", "model": create_cnn_model((32, 32, 3)), "test_images": test_images, "test_labels": test_labels}
    ]

    tpu_required = any(resource[-1] for resource in keywords)
    if tpu_required:
        tpu_available = init_tpu()
        if not tpu_available:
            print("TPU is required but unavailable. Proceeding with fallback to GPU or CPU.")

    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        futures = {}
        for snippet in cnn_snippets:
            resources = (1, 1, 1)  # Example CPU, GPU, TPU allocations
            device = get_device(*resources)
            futures[executor.submit(run_snippet, snippet, device)] = snippet

        for future in concurrent.futures.as_completed(futures):
            snippet = futures[future]
            try:
                result = future.result()
                print(f"Snippet action {snippet['action']} -> {result}")
            except Exception as e:
                print(f"Error executing snippet: {e}")

    total_execution_time = time.time() - start_time
    print(f"Total execution time for code: {total_execution_time:.2f} seconds")

# Main execution
if __name__ == '__main__':
    excel_file_path = 'resource_allocation_dataset.xlsx'  # Update with the correct path
    master(excel_file_path)


Loaded keywords: [('default', 1, 0, 0), ('import', 1, 0, 0), ('CNN', 0, 1, 1), ('RNN', 1, 0, 1), ('preprocess', 1, 1, 0), ('train', 0, 1, 1), ('eval', 1, 0, 0), ('transfer', 0, 1, 1), ('feature', 1, 1, 0), ('tune', 1, 0, 0), ('grid', 1, 0, 0), ('LSTM', 1, 0, 1), ('data_augmentation', 1, 1, 0), ('batch_normalization', 0, 1, 1), ('dropout', 1, 0, 0), ('optimization', 1, 1, 1), ('model_saving', 1, 0, 0), ('hyperparameter_search', 1, 1, 0), ('data_loading', 1, 1, 0), ('evaluation_metrics', 1, 0, 0), ('fine_tuning', 0, 1, 1), ('ensemble_learning', 1, 1, 0), ('tokenization', 1, 0, 0), ('backpropagation', 1, 0, 0), ('regularization', 1, 0, 0), ('transfer_learning', 0, 1, 1), ('seq2seq', 1, 0, 1), ('attention_mechanism', 0, 1, 1), ('model_export', 1, 0, 0), ('real_time_inference', 0, 1, 1), ('predictive_analysis', 1, 0, 0), ('class_weighting', 1, 1, 0), ('grid_search_cv', 1, 0, 0), ('cross_validation', 1, 1, 0), ('feature_scaling', 1, 1, 0), ('loss_function', 1, 0, 0), ('convolutional_layer', 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


TPU initialization failed: Please provide a TPU Name to connect to.
TPU is required but unavailable. Proceeding with fallback to GPU or CPU.
Executing snippet on /device:GPU:0
Executing snippet on /device:GPU:0
Executing snippet on /device:GPU:0
Error executing snippet: You must call `compile()` before using the model.
Snippet action create_model -> <Sequential name=sequential, built=True>
Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.3581 - loss: 1.7411
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5843 - loss: 1.1705
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6481 - loss: 0.9899
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6955 - loss: 0.8696
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7250 - loss: 0.7839
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7450 - loss: 0.7230
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7579 - loss

In [9]:
import time
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

def create_cnn_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

def compile_and_train_model(model, train_images, train_labels, epochs=10):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_images, train_labels, epochs=epochs)

def evaluate_model(model, test_images, test_labels):
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
    print(f'\nTest accuracy: {test_acc}')

def load_and_preprocess_data():
    (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
    train_images = train_images.astype('float32') / 255.0
    test_images = test_images.astype('float32') / 255.0
    return train_images, train_labels, test_images, test_labels

# Measure CNN execution time
start_time = time.time()

# Load data
train_images, train_labels, test_images, test_labels = load_and_preprocess_data()

# Create model
model = create_cnn_model((32, 32, 3))

# Compile and train model
compile_and_train_model(model, train_images, train_labels, epochs=10)  # Adjust epochs as needed

# Evaluate model
evaluate_model(model, test_images, test_labels)

# Print execution time
cnn_execution_time = time.time() - start_time
print(f"Total execution time for CNN code: {cnn_execution_time:.2f} seconds")

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.3368 - loss: 1.7774
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.5645 - loss: 1.2193
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6247 - loss: 1.0554
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6705 - loss: 0.9353
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6967 - loss: 0.8733
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7174 - loss: 0.8022
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7388 - loss: 0.7510
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7480 - loss: 0.7154
Epoch 9/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7688 - loss: 0.6611
Epoch 10/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7823 - loss: 0.6224
313/313 - 1s - 4ms/step - accuracy: 0.7161 - loss: 0.8526

Test accuracy: 0.7160999774932861
Total execution time for C